<a href="https://colab.research.google.com/github/vmatiasw/modelos_y_simulacion/blob/main/P4E6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
from collections import Counter
import random
import time
import numpy as np
import math

In [48]:
def medir_tiempo(func):
    inicio = time.perf_counter()
    resultado = func()
    fin = time.perf_counter()
    return resultado, fin - inicio

def calcular_TVD(dict_fpm_estimada, dict_fpm_real): # Distancia Total de Variación (TVD) entre dos distribuciones discretas.
  return 0.5 * sum(abs(dict_fpm_real[x] - dict_fpm_estimada.get(x, 0)) for x in dict_fpm_real)

def calcular_momentos(fpm):
    esperanza = sum(x * p for x, p in fpm.items())
    varianza = sum((x - esperanza)**2 * p for x, p in fpm.items())
    desviacion = varianza**0.5
    return esperanza, varianza, desviacion

def comparar_distribuciones_df(dict_fpm_estimada, dict_fpm_correcta):
    data_df = pd.DataFrame({
        "Datos": [f"P({k})" for k in dict_fpm_estimada.keys()],
        "Real": [dict_fpm_correcta[x] for x in dict_fpm_estimada.keys()],
        "Estimación": list(dict_fpm_estimada.values()),
        "Diferencia": [abs(dict_fpm_correcta[x] - dict_fpm_estimada[x]) for x in dict_fpm_estimada.keys()]
    })

    esp_real, var_real, desv_real = calcular_momentos(dict_fpm_correcta)
    esp_est, var_est, desv_est = calcular_momentos(dict_fpm_estimada)

    resumen_df = pd.DataFrame([
        {"Datos": "Esperanza", "Real": esp_real, "Estimación": esp_est, "Diferencia": abs(esp_real - esp_est)},
        {"Datos": "Varianza", "Real": var_real, "Estimación": var_est, "Diferencia": abs(var_real - var_est)},
        {"Datos": "Desv. Est.", "Real": desv_real, "Estimación": desv_est, "Diferencia": abs(desv_real - desv_est)},
    ])
    return pd.concat([data_df, resumen_df], ignore_index=True)

def calificar_generador(generador_de_muestras, dict_fpm_correcta, n=100_000):
  soporte = dict_fpm_correcta.keys()

  muestras, tiempos = zip(*[medir_tiempo(generador_de_muestras) for _ in range(n)])

  frecuencia_muestras = Counter(muestras)
  dict_fpm_estimada = {x: frecuencia_muestras.get(x, 0) / n for x in soporte}

  compracion_df = comparar_distribuciones_df(dict_fpm_estimada, dict_fpm_correcta)
  tvd = calcular_TVD(dict_fpm_estimada, dict_fpm_correcta)
  tiempo_promedio = np.mean(tiempos)

  print(compracion_df)
  print()
  print(f"Distancia total de variación: {tvd:.6f}")
  print(f"Tiempo promedio de ejecución: {tiempo_promedio} segundos")


In [79]:
DICT_FPM = {i+1: p for i, p in enumerate([0.15, 0.20, 0.10, 0.35, 0.20])}
SOPORTE = list(DICT_FPM.keys())
print('funcion de probabilidad de masa: ',DICT_FPM)
print('debe dar 1: ',sum(DICT_FPM.values()))

funcion de probabilidad de masa:  {1: 0.15, 2: 0.2, 3: 0.1, 4: 0.35, 5: 0.2}
debe dar 1:  1.0


In [50]:
# I) Describir mediante un pseudocódigo un algoritmo que simule X utilizando el
# método de la transformada inversa y que minimice el número esperado de búsquedas.

def ordenar_dict_fpm(dict_fpm):
  return dict(sorted(dict_fpm.items(), key=lambda x: x[1], reverse=True))

def calcular_dict_fda(dict_fpm):
  """
  Recibe una funcion de probabilidad de masa en forma de diccionario de pares
  (valor, probabilidad) y devuelve su funcion de distribucion acumulada.
  Solo definidas para el soporte.
  """
  F = {}
  suma = 0
  for valor, probabilidad in dict_fpm.items():
    suma += probabilidad
    F[valor] = suma
  return F

def TI(dict_fda):
  u = random.random()
  for v, p in dict_fda.items():
    if u < p:
      return v

print('Sin ordenar:')
dict_fda = calcular_dict_fda(DICT_FPM)
generador_de_muestras = lambda: TI(dict_fda)
calificar_generador(generador_de_muestras, DICT_FPM)
print()
print('Ordenando:')
dict_fda = calcular_dict_fda(ordenar_dict_fpm(DICT_FPM))
generador_de_muestras = lambda: TI(dict_fda)
calificar_generador(generador_de_muestras, DICT_FPM)

Sin ordenar:

        Datos      Real  Estimación  Diferencia
0        P(1)  0.150000    0.151500    0.001500
1        P(2)  0.200000    0.200210    0.000210
2        P(3)  0.100000    0.098400    0.001600
3        P(4)  0.350000    0.350750    0.000750
4        P(5)  0.200000    0.199140    0.000860
5   Esperanza  3.250000    3.245820    0.004180
6    Varianza  1.887500    1.893093    0.005593
7  Desv. Est.  1.373863    1.375897    0.002034

Distancia total de variación: 0.002460
Tiempo promedio de ejecución: 1.3618944399013344e-06 segundos

Ordenando:

        Datos      Real  Estimación  Diferencia
0        P(1)  0.150000    0.149630    0.000370
1        P(2)  0.200000    0.200770    0.000770
2        P(3)  0.100000    0.100600    0.000600
3        P(4)  0.350000    0.349570    0.000430
4        P(5)  0.200000    0.199430    0.000570
5   Esperanza  3.250000    3.248400    0.001600
6    Varianza  1.887500    1.884877    0.002623
7  Desv. Est.  1.373863    1.372908    0.000955

Distan

In [85]:
# II) Describir mediante un pseudocódigo un algoritmo que simule X utilizando el método de aceptación y
# rechazo con una variable soporte Y con distribución binomial B(4,0.45).
N_BIN = 4
P_BIN = 0.45

def crear_dict_fpm(soporte, p):
  return {v: p(v) for v in soporte}

def adecuar_soporte(dict_fpm_objetivo, dict_fpm_modelo):
    soporte_modelo = dict_fpm_modelo.keys()
    return {nuevo_k: v for nuevo_k, v in zip(soporte_modelo, dict_fpm_objetivo.values())}

def fpm_binomial(k, n=N_BIN, p=P_BIN):
    if not 0 <= k <= n:
        return 0.0
    coef = math.comb(n, k)
    prob = coef * (p ** k) * ((1 - p) ** (n - k))
    return prob

_dict_fpm_binomial = crear_dict_fpm(list(range(N_BIN+1)), fpm_binomial)
dict_fpm_binomial = adecuar_soporte(_dict_fpm_binomial, DICT_FPM)

print('debe dar 1: ',sum(dict_fpm_binomial.values())) # da cerca por la precision flotante de python
print()
print(f'X~B({N_BIN},{P_BIN})')
for i in range(1,6):
  print(f'P(X={i}) = ', dict_fpm_binomial[i])

print()
print('funcion de probabilidad de masa de p: ',DICT_FPM)

debe dar 1:  1.0000000000000002

X~B(4,0.45)
P(X=1) =  0.09150625000000003
P(X=2) =  0.2994750000000001
P(X=3) =  0.3675375000000001
P(X=4) =  0.20047500000000004
P(X=5) =  0.04100625

funcion de probabilidad de masa de p:  {1: 0.15, 2: 0.2, 3: 0.1, 4: 0.35, 5: 0.2}


In [86]:
# Método de rechazo con una uniforme discreta, buscando la cota c más baja posible
# con las probabilidades dadas.

C_posibles = [DICT_FPM[i] / dict_fpm_binomial[i] for i in range(1,6)]
C = min(C_posibles)

print(C_posibles)
print('C = ',C)

[1.6392322928761691, 0.6678353785791801, 0.2720810801618882, 1.745853597705449, 4.877305288827923]
C =  0.2720810801618882


In [87]:
def AyR(p, q, c=C):
  """
  Método de aceptación-rechazo para generar muestras de la distribución `p` usando una distribución propuesta `q` y una constante `c`.

  Parámetros:
  p (list/dict): Distribución de probabilidad, donde `p[y]` es la probabilidad asociada a un valor `y`.
  q (function): Función que genera un valor aleatorio `y` según la distribución propuesta.
  c (float): Constante de escala que ajusta la relación entre `p` y `q`.

  Devuelve:
  int: Un valor `y` del soporte de `p` basado en la probabilidad ajustada por `q` y `c`.
  """
  while True:
    u = random.random()
    y = q()
    qy = dict_fpm_binomial[y]
    py = p[y]
    if u < py/(qy*c):
      return y

def q():
  u = random.random()
  indice_random = int(u*len(SOPORTE))
  return SOPORTE[indice_random]

generador_de_muestras = lambda: AyR(DICT_FPM, q)
calificar_generador(generador_de_muestras, DICT_FPM)


        Datos      Real  Estimación  Diferencia
0        P(1)  0.150000    0.198740    0.048740
1        P(2)  0.200000    0.200580    0.000580
2        P(3)  0.100000    0.200400    0.100400
3        P(4)  0.350000    0.199530    0.150470
4        P(5)  0.200000    0.200750    0.000750
5   Esperanza  3.250000    3.002970    0.247030
6    Varianza  1.887500    1.998061    0.110561
7  Desv. Est.  1.373863    1.413528    0.039665

Distancia total de variación: 0.150470
Tiempo promedio de ejecución: 5.659271889362572e-07 segundos
